In [24]:
# --- Centralized Pipeline Configuration ---
# This configuration drives the entire ETL process, making it highly automated.
# Add new tables here to extend the pipeline.
pipeline_config = {
    "customers": {
        "bronze_path": "Files/customers.csv",
        "silver_rules": {
            "primary_keys": ["customer_id"],
            "cast_cols": {"customer_zip_code_prefix": "integer"},
            "trim_cols": ["customer_city", "customer_state","Customer_Name"],
            "timestamp_cols": [],
            "replace_cols": {}
        },
        "dim_keys": ["customer_id", "customer_city", "customer_state"]
    },
    "products": {
        "bronze_path": "Files/products.csv",
        "silver_rules": {
            "primary_keys": ["product_id"],
            "cast_cols": {
                "product_name_lenght": "integer",
                "product_description_lenght": "integer",
                "product_photos_qty": "integer",
                "product_weight_g": "integer",
                "product_length_cm": "integer",
                "product_height_cm": "integer",
                "product_width_cm": "integer"
            },
            "trim_cols": ["product_category_name"],
            "timestamp_cols": [],
            "replace_cols": {}
        },
        "dim_keys": ["product_id", "product_category_name"]
    },

    "orders": {
        "bronze_path": "Files/orders.csv",
        "silver_rules": {
            "primary_keys": ["order_id"],
            "cast_cols": {},
            "trim_cols": [],
            "timestamp_cols": [
                "order_purchase_timestamp",
                "order_approved_at",
                "order_delivered_carrier_date",
                "order_delivered_customer_date",
                "order_estimated_delivery_date"
            ],
            "replace_cols": {}
        },
        "dim_keys": [] # Not a dimension table
    },
    "order_items": {
        "bronze_path": "Files/order_items.csv",
        "silver_rules": {
            "primary_keys": ["order_id","product_id", "product_id"],
            "cast_cols": {
                "price": "double",
                "freight_value": "double"
            },
            "trim_cols": [],
            "timestamp_cols": ["shipping_limit_date"],
            "replace_cols": {}
        },
        "dim_keys": [] # Not a dimension table
    },
    "payments": {
        "bronze_path": "Files/payments.csv",
        "silver_rules": {
            "primary_keys": ["order_id", "payment_sequential"],
            "cast_cols": {
                "payment_sequential": "integer",
                "payment_installments": "integer",
                "payment_value": "double"
            },
            "trim_cols": [],
            "timestamp_cols": [],
            "replace_cols": {"payment_type": ("_", " ")}
        },
        "dim_keys": [] # Not a dimension table
    }
    # Add other tables like "geolocation" or "order_reviews" here with their respective configs
}





StatementMeta(, 326e873b-2ed5-4d16-a226-fa127a8848af, 26, Finished, Available, Finished)

In [25]:

# ---------------------------------------------
# STEP 1: Load Raw CSVs into Bronze Delta Tables (Automated)
# ---------------------------------------------

print("--- STEP 1: Loading Raw CSVs into Bronze Delta Tables ---")
for table_name, config in pipeline_config.items():
    path = config["bronze_path"]
    try:
        print(f"Attempting to read {table_name} from {path}...")
        df = spark.read.option("header", "true").option("inferSchema", "true").csv(path)

        #  Raising an error if the DataFrame is empty
        if df.count() == 0:
            raise ValueError(f"Error: The CSV file for {table_name} at {path} is empty. Cannot process an empty file.")

        df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(f"bronze_{table_name}")
        print(f"Successfully loaded {table_name} into bronze_{table_name} Delta table.")

    except Exception as e:
        print(f"Error processing {table_name} from {path}: {e}")
        print("Please check the file path, file existence, and permissions.")




StatementMeta(, 326e873b-2ed5-4d16-a226-fa127a8848af, 27, Finished, Available, Finished)

--- STEP 1: Loading Raw CSVs into Bronze Delta Tables ---
Attempting to read customers from Files/customers.csv...
Successfully loaded customers into bronze_customers Delta table.
Attempting to read products from Files/products.csv...
Successfully loaded products into bronze_products Delta table.
Attempting to read orders from Files/orders.csv...
Successfully loaded orders into bronze_orders Delta table.
Attempting to read order_items from Files/order_items.csv...
Successfully loaded order_items into bronze_order_items Delta table.
Attempting to read payments from Files/payments.csv...
Successfully loaded payments into bronze_payments Delta table.


In [26]:
# ---------------------------------------------
# STEP 2: Silver Layer - Automated Cleaning and Deduplication
# Silver processing rules defined in config file
# ---------------------------------------------

print("\n--- STEP 2: Creating Silver Layer (Automated Cleaning and Deduplication) ---")
from pyspark.sql.functions import col, to_timestamp, trim, regexp_replace

def process_silver_table(bronze_table_name, silver_table_name, rules):
    """
    Reads data from a bronze table, applies cleaning rules, and writes to a silver table.
    """
    try:
        print(f"Processing {bronze_table_name} for {silver_table_name}")
        df = spark.table(bronze_table_name)

        # Deduplication
        if rules.get("primary_keys"):
            df = df.dropDuplicates(rules["primary_keys"])

        # Type Casting
        for col_name, data_type in rules.get("cast_cols", {}).items():
            if col_name in df.columns:
                df = df.withColumn(col_name, col(col_name).cast(data_type))

        # Trimming
        for col_name in rules.get("trim_cols", []):
            if col_name in df.columns:
                df = df.withColumn(col_name, trim(col(col_name)))

        # Timestamp Conversion
        for col_name in rules.get("timestamp_cols", []):
            if col_name in df.columns:
                df = df.withColumn(col_name, to_timestamp(col(col_name)))

        # Regex Replacement
        for col_name, (pattern, replacement) in rules.get("replace_cols", {}).items():
            if col_name in df.columns:
                df = df.withColumn(col_name, regexp_replace(col(col_name), pattern, replacement))

        df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(silver_table_name)
        print(f"Successfully created {silver_table_name}.")
    except Exception as e:
        print(f"Error processing {bronze_table_name} -> {silver_table_name}: {e}")

# Iterate through the configuration and process each table for the Silver layer
for table_name, config in pipeline_config.items():
    bronze_name = f"bronze_{table_name}"
    silver_name = f"silver_{table_name}"
    process_silver_table(bronze_name, silver_name, config["silver_rules"])




StatementMeta(, 326e873b-2ed5-4d16-a226-fa127a8848af, 28, Finished, Available, Finished)


--- STEP 2: Creating Silver Layer (Automated Cleaning and Deduplication) ---
Processing bronze_customers for silver_customers
Successfully created silver_customers.
Processing bronze_products for silver_products
Successfully created silver_products.
Processing bronze_orders for silver_orders
Successfully created silver_orders.
Processing bronze_order_items for silver_order_items
Successfully created silver_order_items.
Processing bronze_payments for silver_payments
Successfully created silver_payments.


In [27]:
%%sql
drop table if exists dim_date;

StatementMeta(, 326e873b-2ed5-4d16-a226-fa127a8848af, 29, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [28]:
# ---------------------------------------------
# STEP 3: Generate dim_date Table
# ---------------------------------------------
from pyspark.sql.functions import expr, sequence, to_date, col, year, month, dayofweek, dayofmonth, weekofyear, quarter, dayofyear, date_format
date_range = spark.sql("SELECT sequence(to_date('2016-01-01'), to_date('2025-12-31'), interval 1 day) as dates")
dim_date = date_range.selectExpr("explode(dates) as full_date") \
                     .withColumn("year", year("full_date")) \
                     .withColumn("month", month("full_date")) \
                     .withColumn("day_of_month", dayofmonth("full_date")) \
                     .withColumn("day_of_week", dayofweek("full_date")) \
                     .withColumn("week_of_year", weekofyear("full_date")) \
                     .withColumn("quarter", quarter("full_date")) \
                     .withColumn("day_of_year", dayofyear("full_date")) \
                     .withColumn("month_name", date_format(col("full_date"), "MMMM")) \
                     .withColumn("day_name", date_format(col("full_date"), "EEEE"))

dim_date.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("dim_date")
print("Successfully created dim_date.")




StatementMeta(, 326e873b-2ed5-4d16-a226-fa127a8848af, 30, Finished, Available, Finished)

Successfully created dim_date.


In [29]:
# ---------------------------------------------
# STEP 4: SCD2 Merge Into Logic for Dims (Automated from Silver)
# ---------------------------------------------

print("\n--- STEP 4: Applying SCD2 Merge Logic for Dimensions (from Silver) ---")
from pyspark.sql.functions import current_timestamp, lit, md5, concat_ws

def scd2_merge(table_name, business_keys):
    # Read from the silver table
    df = spark.table(f"silver_{table_name}")
    
    # Add SCD2 specific columns
    df = df.withColumn("sk", md5(concat_ws("||", *business_keys))) \
           .withColumn("effective_start_date", current_timestamp()) \
           .withColumn("effective_end_date", lit(None).cast("timestamp")) \
           .withColumn("is_current", lit(True))

    target_table = f"dim_{table_name}"

    # Define columns to exclude from the dynamic schema generation
    # These columns are explicitly added later in the CREATE TABLE statement
    excluded_cols = ["sk", "effective_start_date", "effective_end_date", "is_current"]
    
    # Filter out the excluded columns from the DataFrame's columns for DDL generation
    # Ensure that the column names are quoted for SQL DDL
    base_columns_for_ddl = [f"`{col}` {df.schema[col].dataType.simpleString().upper()}" for col in df.columns if col not in excluded_cols]

    # Create the dimension table if it doesn't exist
    spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {target_table} (
        sk STRING,
        {', '.join(base_columns_for_ddl)},
        effective_start_date TIMESTAMP,
        effective_end_date TIMESTAMP,
        is_current BOOLEAN
    ) USING DELTA
    """)

    df.createOrReplaceTempView("staging")

    # Perform the SCD Type 2 merge
    spark.sql(f"""
    MERGE INTO {target_table} tgt
    USING staging src
    ON tgt.sk = src.sk AND tgt.is_current = TRUE
    WHEN MATCHED THEN
      UPDATE SET tgt.is_current = FALSE, tgt.effective_end_date = current_timestamp()
    WHEN NOT MATCHED THEN
      INSERT *
    """)
    print(f"Successfully applied SCD2 merge to {target_table}.")

# Iterate through the configuration and apply SCD2 to dimension tables
for table_name, config in pipeline_config.items():
    if config.get("dim_keys"): # Check if dim_keys are defined for this table
        scd2_merge(table_name, config["dim_keys"])
    else:
        print(f"Skipping SCD2 merge for {table_name} as it's not configured as a dimension table.")




StatementMeta(, 326e873b-2ed5-4d16-a226-fa127a8848af, 31, Finished, Available, Finished)


--- STEP 4: Applying SCD2 Merge Logic for Dimensions (from Silver) ---
Successfully applied SCD2 merge to dim_customers.
Successfully applied SCD2 merge to dim_products.
Skipping SCD2 merge for orders as it's not configured as a dimension table.
Skipping SCD2 merge for order_items as it's not configured as a dimension table.
Skipping SCD2 merge for payments as it's not configured as a dimension table.


In [30]:
# ---------------------------------------------
# STEP 5: Build Accumulating Snapshot Fact with Surrogate Keys (Reading from Silver)
# ---------------------------------------------

print("\n--- STEP 5: Building Accumulating Snapshot Fact with Surrogate Keys ---")
from pyspark.sql import functions as F

# Read from silver tables
orders = spark.table("silver_orders")
items = spark.table("silver_order_items")
payments = spark.table("silver_payments")

# Read from gold dimensions (which are now populated from silver)
dim_customers = spark.table("dim_customers").filter("is_current = TRUE")
dim_products = spark.table("dim_products").filter("is_current = TRUE")

# Join and derive fact metrics
# Use common column names directly after joins.
# For columns with potential ambiguity or from specific tables, use aliases like 'items.price'
fact_df = orders.alias("orders").join(items.alias("items"), "order_id", "left")  \
    .join(payments.alias("payments"), "order_id", "left") \
    .join(dim_customers.select("customer_id", "sk").withColumnRenamed("sk", "customer_sk"), "customer_id", "left") \
    .join(dim_products.select("product_id", "sk").withColumnRenamed("sk", "product_sk"), "product_id", "left")  \
     .selectExpr(
        "order_id"  , # Directly use order_id as it's the join key and unambiguous
        "customer_sk" ,
         "product_sk" ,
         "order_purchase_timestamp" ,
         "order_delivered_carrier_date" ,
         "order_delivered_customer_date" ,
         "payment_type" ,
         "items.price + items.shipping_charges as total_price" , # Specify table alias for clarity
         "payments.payment_value", # Specify table alias for clarity
         "datediff(order_delivered_carrier_date, order_purchase_timestamp) as days_to_ship",
         "datediff(order_delivered_customer_date, order_delivered_carrier_date) as days_to_deliver"
   
    )

fact_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("fact_order_lifecycle")
print("Successfully created fact_order_lifecycle.")




StatementMeta(, 326e873b-2ed5-4d16-a226-fa127a8848af, 32, Finished, Available, Finished)


--- STEP 5: Building Accumulating Snapshot Fact with Surrogate Keys ---
Successfully created fact_order_lifecycle.


In [31]:
%%sql
select * from fact_order_lifecycle limit 10

StatementMeta(, 326e873b-2ed5-4d16-a226-fa127a8848af, 33, Finished, Available, Finished)

<Spark SQL result set with 10 rows and 11 fields>

In [34]:
# ---------------------------------------------
# STEP 6: Gold View for Reporting (Power BI or DuckDB)
# ---------------------------------------------
print("\n--- STEP 6: Creating Gold View for Reporting ---")
spark.sql("""
CREATE OR REPLACE VIEW gold_order_summary AS
SELECT
  f.order_id,
  d.Customer_Name,
    d.customer_city,
  p.product_category_name,
  f.order_purchase_timestamp AS order_date,
  f.total_price,
  f.days_to_ship,
  f.days_to_deliver

FROM fact_order_lifecycle f
LEFT JOIN dim_customers d ON f.customer_sk = d.sk AND d.is_current = TRUE
LEFT JOIN dim_products p ON f.product_sk = p.sk AND p.is_current = TRUE
WHERE f.days_to_deliver IS NOT NULL
""")
print("Successfully created gold_order_summary view.")




StatementMeta(, 326e873b-2ed5-4d16-a226-fa127a8848af, 36, Finished, Available, Finished)


--- STEP 6: Creating Gold View for Reporting ---
Successfully created gold_order_summary view.


In [36]:
%%sql
select   * from gold_order_summary where Customer_Name is not null limit 10;

StatementMeta(, 326e873b-2ed5-4d16-a226-fa127a8848af, 38, Finished, Available, Finished)

<Spark SQL result set with 10 rows and 8 fields>